# Sum prod
https://en.wikipedia.org/wiki/Sum_and_Product_Puzzle:

X and Y are two different whole numbers greater than 1. Their sum is not greater than 100, and Y is greater than X. S and P are two mathematicians (and consequently perfect logicians); S knows the sum X + Y and P knows the product X × Y. Both S and P know all the information in this paragraph.

The following conversation occurs:

    S says "P does not know X and Y."
    P says "Now I know X and Y."
    S says "Now I also know X and Y."

What are X and Y? 

In [19]:
import pandas as pd
import numpy as np

In [1]:
all_pairs = []
for i in range(2, 100):
    for j in range(i + 1, 100):
        if i + j <= 100:
            all_pairs.append({"x": i, "y": j})

In [3]:
len(all_pairs)

2352

In [4]:
for p in all_pairs:
    p["s"] = p["x"] + p["y"]
    p["p"] = p["x"] * p["y"]

In [58]:
df = pd.DataFrame(all_pairs)

In [65]:
pos_prod = df.groupby(["p"], as_index = False).s.count().rename(columns = {"s":"num_pairs_prod"})

In [68]:
pos_prod["pos_prod"] = pos_prod.num_pairs_prod.map(lambda x: 1 if x > 1 else 0)

In [69]:
pos_prod

,p,num_pairs_prod,pos_prod
0,6,1,0
1,8,1,0
2,10,1,0
3,12,2,1
4,14,1,0
...,...,...,...
1174,2475,1,0
1175,2484,1,0
1176,2491,1,0
1177,2496,1,0


In [70]:
df_pos_prod = df.merge(right = pos_prod, on = ["p"], how = "left")

In [71]:
df_pos_prod.head()

,x,y,s,p,num_pairs_prod,pos_prod
0,2,3,5,6,1,0
1,2,4,6,8,1,0
2,2,5,7,10,1,0
3,2,6,8,12,2,1
4,2,7,9,14,1,0


In [76]:
pos_sum = df.groupby(["s"], as_index = False).p.count().rename(columns = {"p":"num_pairs_sum"})

In [77]:
pos_sum["pos_sum"] = pos_sum.num_pairs_sum.map(lambda x: 1 if x > 1 else 0)

In [78]:
pos_sum.head()

,s,num_pairs_sum,pos_sum
0,5,1,0
1,6,1,0
2,7,2,1
3,8,2,1
4,9,3,1


In [79]:
df_pos_sum = df_pos_prod.merge(right = pos_sum, on = "s", how = "left")

In [80]:
df_pos_sum.head()

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum
0,2,3,5,6,1,0,1,0
1,2,4,6,8,1,0,1,0
2,2,5,7,10,1,0,2,1
3,2,6,8,12,2,1,2,1
4,2,7,9,14,1,0,3,1


In [81]:
pos_pos_sum = df_pos_sum.groupby(["s"], as_index = False).pos_prod.min().rename(columns = {"pos_prod":"pos_pos_sum"})

In [85]:
df_pos_pos_sum = df_pos_sum.merge(right = pos_pos_sum, on = ["s"], how = "left")

In [86]:
df_pos_pos_sum.head()

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum
0,2,3,5,6,1,0,1,0,0
1,2,4,6,8,1,0,1,0,0
2,2,5,7,10,1,0,2,1,0
3,2,6,8,12,2,1,2,1,0
4,2,7,9,14,1,0,3,1,0


In [90]:
pos_pos_prod = df_pos_pos_sum.groupby(["p"], as_index = False).pos_pos_sum.agg(lambda x: 1 if sum(x) == 1 else 0).rename(columns = {"pos_pos_sum": "pos_pos_prod"})

In [91]:
df_pos_pos_prod = df_pos_pos_sum.merge(right = pos_pos_prod, on = ["p"], how = "left")

In [93]:
df_pos_pos_prod[df_pos_pos_prod.pos_pos_prod == 1]

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum,pos_pos_prod
6,2,9,11,18,2,1,4,1,1,1
9,2,12,14,24,3,1,5,1,0,1
11,2,14,16,28,2,1,6,1,0,1
22,2,25,27,50,2,1,12,1,1,1
23,2,26,28,52,2,1,12,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1602,23,30,53,690,3,1,25,1,1,1
1653,24,28,52,672,6,1,24,1,0,1
1654,24,29,53,696,3,1,25,1,1,1
1704,25,28,53,700,4,1,25,1,1,1


In [96]:
p3sum = df_pos_pos_prod.groupby(["s"], as_index = False).pos_pos_prod.agg(lambda x: 1 if sum(x) == 1 else 0).rename(columns = {"pos_pos_prod":"p3sum"})

In [97]:
df_p3sum = df_pos_pos_prod.merge(right = p3sum, on = ["s"], how = "left")

In [98]:
df_p3sum.head()

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum,pos_pos_prod,p3sum
0,2,3,5,6,1,0,1,0,0,0,0
1,2,4,6,8,1,0,1,0,0,0,0
2,2,5,7,10,1,0,2,1,0,0,0
3,2,6,8,12,2,1,2,1,0,0,0
4,2,7,9,14,1,0,3,1,0,0,1


In [99]:
df_p3sum[(df_p3sum.pos_prod == 1) & (df_p3sum.pos_sum == 1) &(df_p3sum.pos_pos_sum == 1) &(df_p3sum.pos_pos_prod == 1) &(df_p3sum.p3sum == 1)]

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum,pos_pos_prod,p3sum
198,4,13,17,52,2,1,7,1,1,1,1


In [100]:
df_p3sum[df_p3sum.s == 17]

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum,pos_pos_prod,p3sum
12,2,15,17,30,3,1,7,1,1,0,1
106,3,14,17,42,3,1,7,1,1,0,1
198,4,13,17,52,2,1,7,1,1,1,1
288,5,12,17,60,5,1,7,1,1,0,1
376,6,11,17,66,3,1,7,1,1,0,1
462,7,10,17,70,3,1,7,1,1,0,1
546,8,9,17,72,5,1,7,1,1,0,1


In [101]:
df_p3sum[df_p3sum.p == 52]

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum,pos_pos_prod,p3sum
23,2,26,28,52,2,1,12,1,0,1,0
198,4,13,17,52,2,1,7,1,1,1,1


In [103]:
df_p3sum[df_p3sum.p == 30]

,x,y,s,p,num_pairs_prod,pos_prod,num_pairs_sum,pos_sum,pos_pos_sum,pos_pos_prod,p3sum
12,2,15,17,30,3,1,7,1,1,0,1
102,3,10,13,30,3,1,5,1,0,0,0
282,5,6,11,30,3,1,4,1,1,0,0
